# Análise Exploratória de Dados (EDA)
## Inventory Anomaly Detector

Este notebook realiza a análise exploratória dos dados de estoque e consumo.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configurações
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Bibliotecas importadas com sucesso!")


## 1. Carregamento dos Dados


In [ ]:
# Carregar dados
df = pd.read_csv('data/inventory_data.csv')

# Converter coluna de data
df['data'] = pd.to_datetime(df['data'])

print(f"Shape do dataset: {df.shape}")
print(f"\nPrimeiras linhas:")
df.head()


## 2. Informações Básicas do Dataset


In [ ]:
# Informações gerais
print("=== INFORMAÇÕES DO DATASET ===")
print(f"\nPeríodo dos dados:")
print(f"Data inicial: {df['data'].min()}")
print(f"Data final: {df['data'].max()}")
print(f"Total de dias: {(df['data'].max() - df['data'].min()).days + 1}")

print(f"\nProdutos únicos: {df['produto_id'].nunique()}")
print(f"Produtos: {df['produto_id'].unique()}")

print(f"\nTipos de dados:")
print(df.dtypes)

print(f"\nValores nulos:")
print(df.isnull().sum())


## 3. Estatísticas Descritivas


In [ ]:
# Estatísticas descritivas gerais
print("=== ESTATÍSTICAS DESCRITIVAS ===")
df[['estoque', 'consumo']].describe()


In [ ]:
# Estatísticas por produto
print("=== ESTATÍSTICAS POR PRODUTO ===")
df.groupby('produto_id')[['estoque', 'consumo']].describe()


## 4. Visualizações - Distribuições


In [ ]:
# Distribuições de estoque e consumo
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Histograma de estoque
axes[0, 0].hist(df['estoque'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Distribuição de Estoque')
axes[0, 0].set_xlabel('Estoque')
axes[0, 0].set_ylabel('Frequência')

# Histograma de consumo
axes[0, 1].hist(df['consumo'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_title('Distribuição de Consumo')
axes[0, 1].set_xlabel('Consumo')
axes[0, 1].set_ylabel('Frequência')

# Boxplot de estoque por produto
df.boxplot(column='estoque', by='produto_id', ax=axes[1, 0])
axes[1, 0].set_title('Estoque por Produto')
axes[1, 0].set_xlabel('Produto')
axes[1, 0].set_ylabel('Estoque')

# Boxplot de consumo por produto
df.boxplot(column='consumo', by='produto_id', ax=axes[1, 1])
axes[1, 1].set_title('Consumo por Produto')
axes[1, 1].set_xlabel('Produto')
axes[1, 1].set_ylabel('Consumo')

plt.tight_layout()
plt.show()


## 5. Análise de Séries Temporais


In [ ]:
# Série temporal de estoque e consumo por produto
n_products = df['produto_id'].nunique()
fig, axes = plt.subplots(n_products, 2, figsize=(18, 4*n_products))

if n_products == 1:
    axes = axes.reshape(1, -1)

for idx, produto in enumerate(df['produto_id'].unique()):
    df_produto = df[df['produto_id'] == produto].sort_values('data')
    
    # Estoque ao longo do tempo
    axes[idx, 0].plot(df_produto['data'], df_produto['estoque'], linewidth=1.5)
    axes[idx, 0].set_title(f'Estoque - {produto}')
    axes[idx, 0].set_xlabel('Data')
    axes[idx, 0].set_ylabel('Estoque')
    axes[idx, 0].grid(True, alpha=0.3)
    axes[idx, 0].tick_params(axis='x', rotation=45)
    
    # Consumo ao longo do tempo
    axes[idx, 1].plot(df_produto['data'], df_produto['consumo'], linewidth=1.5, color='orange')
    axes[idx, 1].set_title(f'Consumo - {produto}')
    axes[idx, 1].set_xlabel('Data')
    axes[idx, 1].set_ylabel('Consumo')
    axes[idx, 1].grid(True, alpha=0.3)
    axes[idx, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


## 6. Análise de Correlação


In [ ]:
# Correlação entre estoque e consumo
correlation = df['estoque'].corr(df['consumo'])
print(f"Correlação entre Estoque e Consumo: {correlation:.4f}")

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['consumo'], df['estoque'], alpha=0.5, s=20)
plt.xlabel('Consumo')
plt.ylabel('Estoque')
plt.title(f'Relação entre Consumo e Estoque (Correlação: {correlation:.4f})')
plt.grid(True, alpha=0.3)
plt.show()


## 7. Análise de Sazonalidade


In [ ]:
# Extrair componentes temporais
df['ano'] = df['data'].dt.year
df['mes'] = df['data'].dt.month
df['dia_semana'] = df['data'].dt.dayofweek
df['dia_mes'] = df['data'].dt.day

# Consumo médio por dia da semana
consumo_semanal = df.groupby('dia_semana')['consumo'].mean()
dias = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sáb', 'Dom']

# Consumo médio por mês
consumo_mensal = df.groupby('mes')['consumo'].mean()

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].bar(range(7), consumo_semanal.values, color='skyblue', edgecolor='black')
axes[0].set_xticks(range(7))
axes[0].set_xticklabels(dias)
axes[0].set_title('Consumo Médio por Dia da Semana')
axes[0].set_ylabel('Consumo Médio')
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].bar(range(1, 13), consumo_mensal.values, color='coral', edgecolor='black')
axes[1].set_xticks(range(1, 13))
axes[1].set_title('Consumo Médio por Mês')
axes[1].set_xlabel('Mês')
axes[1].set_ylabel('Consumo Médio')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 8. Identificação de Valores Extremos (Potenciais Anomalias)


In [ ]:
# Usar IQR para identificar outliers
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return (series < lower_bound) | (series > upper_bound)

# Detectar outliers
outliers_estoque = detect_outliers_iqr(df['estoque'])
outliers_consumo = detect_outliers_iqr(df['consumo'])

print(f"Outliers em Estoque: {outliers_estoque.sum()} ({outliers_estoque.sum()/len(df)*100:.2f}%)")
print(f"Outliers em Consumo: {outliers_consumo.sum()} ({outliers_consumo.sum()/len(df)*100:.2f}%)")

# Visualizar outliers
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].boxplot(df['estoque'], vert=True)
axes[0].set_title('Boxplot - Estoque (com outliers)')
axes[0].set_ylabel('Estoque')
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].boxplot(df['consumo'], vert=True)
axes[1].set_title('Boxplot - Consumo (com outliers)')
axes[1].set_ylabel('Consumo')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 9. Resumo e Conclusões


In [ ]:
print("=== RESUMO DA ANÁLISE EXPLORATÓRIA ===\n")
print(f"1. Dataset contém {len(df)} registros")
print(f"2. Período: {df['data'].min().strftime('%Y-%m-%d')} a {df['data'].max().strftime('%Y-%m-%d')}")
print(f"3. {df['produto_id'].nunique()} produtos únicos")
print(f"4. Estoque médio: {df['estoque'].mean():.2f}")
print(f"5. Consumo médio: {df['consumo'].mean():.2f}")
print(f"6. Correlação estoque-consumo: {df['estoque'].corr(df['consumo']):.4f}")
print(f"7. Valores nulos: {df.isnull().sum().sum()}")
print(f"8. Outliers detectados (IQR):")
print(f"   - Estoque: {outliers_estoque.sum()}")
print(f"   - Consumo: {outliers_consumo.sum()}")

print("\n=== PRÓXIMOS PASSOS ===")
print("- Preparar dados para modelagem")
print("- Criar features de série temporal")
print("- Treinar modelos Prophet para previsão")
print("- Aplicar Isolation Forest para detecção de anomalias")
